In [0]:
pip install nltk pandas matplotlib scipy huggingface_hub pyarrow fsspec seaborn scikit-learn wordcloud -q

In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def tokenize(token):
    return nltk.word_tokenize(token, language='english', preserve_line=True)

In [0]:
tokenize("why is this not working?")

In [0]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import os
import nltk
import numpy as np
from scipy.sparse import csr_matrix
# corpus de nltk para 'tokenizer' y 'stopwords'
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')

## preprocesamiento de textos

In [0]:
splits = {'train': 'data/train-00000-of-00001-7b34565378f02992.parquet', 'val': 'data/val-00000-of-00001-d7338c59b5e5031f.parquet', 'test': 'data/test-00000-of-00001-c830a979da438bff.parquet'}
df_train = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["val"])
df_test = pd.read_parquet("hf://datasets/PrevenIA/spanish-suicide-intent/" + splits["test"])

df = pd.concat([df_train, df_val, df_test]).reset_index(drop=True)
df.info()

In [0]:
df["Label"].value_counts()

In [0]:
# Tokenizing the text in each row
df['tokenized_text'] = df['Text'].apply(lambda text: nltk.word_tokenize(str(text), language='spanish', preserve_line=True))

# Counting the number of tokens in each row
df['count_text'] = df['tokenized_text'].apply(len)

In [0]:
df['Label'] = df['Label'].astype('category')
df[['count_text', 'Label']].describe()

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Función para identificar los outliers
def find_outliers(df, column, label_column):
    outliers = pd.DataFrame(columns=df.columns)
    for label in df[label_column].unique():
        subset = df[df[label_column] == label]
        Q1 = subset[column].quantile(0.25)
        Q3 = subset[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers_subset = subset[(subset[column] < lower_bound) | (subset[column] > upper_bound)]
        outliers = pd.concat([outliers, outliers_subset])
    return outliers

# Suponiendo que los datos están en un DataFrame llamado 'df'
# Crear el boxplot sin outliers
plt.figure(figsize=(10, 6))
sns.boxplot(x='Label', y='count_text', data=df, showfliers=False)
plt.title('Boxplot de Palabras en post por etiqueta (sin outliers)')
plt.xlabel('Etiqueta')
plt.ylabel('Palabras por post')
plt.show()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear el boxplot sin outliers
plt.figure(figsize=(10, 6))
ax = sns.boxplot(x='Label', y='count_text', data=df, showfliers=False)

# Calcular la media y la mediana por cada grupo
means = df.groupby('Label')['count_text'].mean()
medians = df.groupby('Label')['count_text'].median()
positions = range(len(means))

# Agregar anotaciones de la media y la mediana
for tick, label in zip(positions, ax.get_xticks()):
    # Obtener la media y la mediana para este grupo
    mean = means.iloc[tick]
    median = medians.iloc[tick]
    
    # Anotar la media
    ax.text(label, mean, f'Media: {mean:.1f}', horizontalalignment='center', verticalalignment='center', color='white', fontsize=10)
    
    # Anotar la mediana
    ax.text(label, median, f'Mediana: {median:.1f}', horizontalalignment='center', verticalalignment='center', color='white', fontsize=10)

# Títulos y etiquetas
plt.title('Boxplot de Palabras en post por etiqueta (sin outliers)')
plt.xlabel('Etiqueta')
plt.ylabel('Palabras por post')

# Mostrar gráfico
plt.show()

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear el gráfico KDE para cada etiqueta
plt.figure(figsize=(10, 6))

# Graficar la distribución (KDE) para cada etiqueta
sns.kdeplot(data=df, x='count_text', hue='Label', fill=True)

# Limitar el rango del eje x para excluir los outliers
plt.xlim(0, df['count_text'].quantile(0.99))  # Limitar al percentil 95

# Títulos y etiquetas
plt.title('Distribuciones de Palabras por Post por Etiqueta (KDE)')
plt.xlabel('Palabras por post')
plt.ylabel('Densidad')

# Mostrar gráfico
plt.show()

In [0]:
# Encontrar los outliers
outliers = find_outliers(df, 'count_text', 'Label')

# Contar el número de outliers por etiqueta
outliers_count = outliers.groupby('Label').size()

# Contar el total de observaciones por etiqueta
total_count = df.groupby('Label').size()

# Crear gráfico separado para los outliers
plt.figure(figsize=(10, 6))
sns.stripplot(x='Label', y='count_text', data=outliers, color='red', marker='o', jitter=True)

# Añadir una etiqueta que muestre el número de outliers y el porcentaje con respecto al total
for label, count in outliers_count.items():
    total = total_count[label]
    percentage = (count / total) * 100
    plt.text(label, outliers['count_text'].max() + 1, f'Outliers: {count} ({percentage:.2f}%)', 
             horizontalalignment='center', color='black', weight='bold')

plt.title('Outliers de count_text por Label')
plt.xlabel('Label')
plt.ylabel('count_text')
plt.show()

In [0]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Combinar todos los textos en un solo string
text = " ".join(review for review in df['Text'])

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Mostrar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [0]:
wordcloud = WordCloud(
    width=800, 
    height=400, 
    max_words=200, 
    background_color='white', 
    colormap='viridis',  # Cambiar el esquema de colores
    stopwords=None,      # Puedes añadir tus propias stopwords
    contour_color='steelblue', 
    contour_width=3
).generate(text)

In [0]:
def unir_tokens_from_lista(lista_tokens:list)->list:
    all_tokens = []
    for t in lista_tokens:
        all_tokens.extend(t)
    print("all_tokens_title =",len(all_tokens))
    return all_tokens 

In [0]:
def analisis_grafico(all_tokens: list, palabras_top: int = 10):
    # Análisis de frecuencia en palabras 
    fdist = nltk.FreqDist(all_tokens)
    print('Size BoW =', len(fdist))
    
    # Palabras más comunes
    topwords = fdist.most_common(palabras_top)
    x, y = zip(*topwords)
    
    # Crear gráfico
    plt.figure(figsize=(15, 10))
    plt.bar(x, y)
    
    # Rotar etiquetas del eje X
    plt.xticks(rotation=90, fontsize=14)  # Aumenta el tamaño de las etiquetas en el eje X
    plt.yticks(fontsize=14)  # Aumenta el tamaño de las etiquetas en el eje Y
    
    # Etiquetas de los ejes con tamaño de fuente aumentado
    plt.xlabel('Palabras', fontsize=16)
    plt.ylabel('Frecuencia', fontsize=16)
    
    # Mostrar gráfico
    plt.show()

In [0]:
import matplotlib.pyplot as plt
import nltk
import seaborn as sns

def analisis_grafico_2(all_tokens: list, palabras_top: int = 10):
    # Análisis de frecuencia en palabras
    fdist = nltk.FreqDist(all_tokens)
    print('Size BoW =', len(fdist))
    
    # Palabras más comunes
    topwords = fdist.most_common(palabras_top)
    x, y = zip(*topwords)
    
    # Crear gráfico
    plt.figure(figsize=(15, 10))
    
    # Usar una paleta de colores de seaborn
    palette = sns.color_palette("husl", palabras_top)  # Cambia "husl" por otra paleta si lo prefieres
    plt.bar(x, y, color=palette)
    
    # Rotar etiquetas del eje X
    plt.xticks(rotation=90, fontsize=14)
    plt.yticks(fontsize=14)
    
    # Etiquetas de los ejes
    plt.xlabel('Palabras', fontsize=16)
    plt.ylabel('Frecuencia', fontsize=16)
    
    # Mostrar gráfico
    plt.show()

In [0]:

def analisis_grafico_3(all_tokens: list, palabras_top: int = 10):
    # Análisis de frecuencia en palabras
    fdist = nltk.FreqDist(all_tokens)
    print('Size BoW =', len(fdist))
    
    # Palabras más comunes
    topwords = fdist.most_common(palabras_top)
    x, y = zip(*topwords)
    
    # Crear gráfico de barras horizontales
    plt.figure(figsize=(15, 10))
    
    # Usar un degradado de azul en la paleta de colores
    palette = sns.light_palette("blue", reverse=True, n_colors=palabras_top)  # Paleta en degradado azul
    
    # Graficar barras horizontales con el degradado
    plt.barh(x, y, color=palette)
    
    # Invertir el eje Y para que el ranking se vea de mayor a menor
    plt.gca().invert_yaxis()
    
    # Ajustar el tamaño de las etiquetas
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
    # Etiquetas de los ejes
    plt.xlabel('Frecuencia', fontsize=16)
    plt.ylabel('Palabras', fontsize=16)
    
    # Mostrar gráfico
    plt.show()

In [0]:
lista_intencion_suicidio = df[df["Label"]==1]["tokenized_text"].values
all_tokens = unir_tokens_from_lista(lista_intencion_suicidio)
analisis_grafico_3(all_tokens)


### preprocesamiento inicial y elimininacion de stop words

In [0]:
from nltk.corpus import stopwords
def quitar_tildes(s):  
    s = re.sub(r'[áàä]', 'a', s)  
    s = re.sub(r'[éèë]', 'e', s)  
    s = re.sub(r'[íìï]', 'i', s)  
    s = re.sub(r'[óòö]', 'o', s)  
    s = re.sub(r'[úùü]', 'u', s)  
    return s  

def preprocesamiento_and_stopwords(all_tokens):
    stop_words_nltk = set(stopwords.words('spanish'))
    prepro_tokens = [quitar_tildes(w) for w in all_tokens]
    prepro_tokens = [w.lower() for w in prepro_tokens]  
    prepro_tokens = [re.sub(r'[^A-Za-z0-9]+','',w) for w in prepro_tokens]
    prepro_tokens = [w for w in prepro_tokens if w not in stop_words_nltk]
    prepro_tokens = [w for w in prepro_tokens if w.isalpha()]
    prepro_tokens = [w for w in prepro_tokens if len(w)>3]
    return prepro_tokens

In [0]:
prepro_tokens = preprocesamiento_and_stopwords(all_tokens)
analisis_grafico_3(prepro_tokens)

In [0]:
# Combinar todos los textos en un solo string
text = " ".join(prepro_tokens)

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Mostrar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### lematizacion y stemmer

In [0]:
#Lemmatization y stemmer

from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer

def preprocesamiento_lematizacion_stemming(tokens):
    lancaster = LancasterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()

    prepro_tokens = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in tokens ]
    prepro_tokens = [wordnet_lemmatizer.lemmatize(w, pos="n") for w in prepro_tokens ]
    prepro_tokens = [wordnet_lemmatizer.lemmatize(w, pos="a") for w in prepro_tokens ]
    prepro_tokens = [wordnet_lemmatizer.lemmatize(w, pos="r") for w in prepro_tokens ]

    prepro_tokens = [lancaster.stem(w) for w in prepro_tokens]
    prepro_tokens = [w for w in prepro_tokens if len(w)>2]

    return prepro_tokens


In [0]:
prepro_tokens = preprocesamiento_lematizacion_stemming(prepro_tokens)
analisis_grafico(prepro_tokens)

In [0]:
# Combinar todos los textos en un solo string
text = " ".join(prepro_tokens)

# Crear la nube de palabras
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Mostrar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Matriz de frecuencia de terminos

In [0]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def crear_matriz_frecuencia_terminos(prepro_tokens: list):
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=None)
    
    # Matriz dispersa de frecuencia de términos
    X = vectorizer.fit_transform(prepro_tokens)
    
    # Obtener el vocabulario
    T = vectorizer.get_feature_names_out()

    return X, T

def extraer_topicos_con_LDA(X, T, numero_topicos=5, numero_terminos=10):
    lda = LatentDirichletAllocation(n_components=numero_topicos, random_state=42)
    lda.fit(X)

    # Mostrar los términos más importantes por cada tópico
    for idx, topic in enumerate(lda.components_):
        print(f"Tópico {idx + 1}:")
        top_terminos = [T[i] for i in topic.argsort()[:-numero_terminos - 1:-1]]
        print(" ".join(top_terminos))


X, T = crear_matriz_frecuencia_terminos(prepro_tokens)
extraer_topicos_con_LDA(X, T, numero_topicos=5, numero_terminos=5)

In [0]:
extraer_topicos_con_LDA(X, T, numero_topicos=5, numero_terminos=10)

In [0]:
lista_intencion_suicidio = df["tokenized_text"].values
all_tokens = unir_tokens_from_lista(lista_intencion_suicidio)

In [0]:
prepro_tokens = preprocesamiento_and_stopwords(all_tokens)

In [0]:
len(prepro_tokens)

In [0]:
X, T = crear_matriz_frecuencia_terminos(prepro_tokens)
len(T)

In [0]:
extraer_topicos_con_LDA(X, T, numero_topicos=5, numero_terminos=10)